load hf model -> test ctx + calc sliding window pplx -> linear scaling/ntk scaling -> calc pplx

In [1]:
!pip3 install -q huggingface_hub transformers torch tqdm datasets

In [3]:
from huggingface_hub import login, snapshot_download
# login()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# !huggingface-cli login # do it in terminal

In [4]:
local_dir = snapshot_download("google/gemma-1.1-2b-it")

Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 49030.12it/s]


In [5]:
local_dir

'/root/.cache/huggingface/hub/models--google--gemma-1.1-2b-it/snapshots/bf4924f313df5166dee1467161e886e55f2eb4d4'

In [6]:
!ls -a '/root/.cache/huggingface/hub/models--google--gemma-1.1-2b-it/snapshots/bf4924f313df5166dee1467161e886e55f2eb4d4'

.		generation_config.json		  tokenizer.json
..		model-00001-of-00002.safetensors  tokenizer.model
.gitattributes	model-00002-of-00002.safetensors  tokenizer_config.json
README.md	model.safetensors.index.json
config.json	special_tokens_map.json


In [21]:
!pip3 install -q accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import torch
from tqdm import tqdm
from datasets import load_dataset

In [9]:
device = "cuda"

In [11]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In circuits of code, a mind takes form,


In [12]:
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt").to(device)

do scaling: linear/ntk etc

In [23]:
from transformers.models.gemma.modeling_gemma import GemmaRotaryEmbedding 

old_init = GemmaRotaryEmbedding.__init__

In [66]:
# def linear_scaled_init(self, dim, max_position_embeddings=2048, base=10000, device=None):
#     old_init(self, dim, max_position_embeddings, base, device)

# # GemmaRotaryEmbedding.__init__ = linear_scaled_init
# setattr(GemmaRotaryEmbedding, '__init__', linear_scaled_init)

In [67]:
# def linear_scaled_forward(self, x, position_ids, seq_len=None):
#     scaling_factor = 4.0
#     position_ids = position_ids.float() / scaling_factor
#     cos, sin = super(GemmaRotaryEmbedding, self).forward(x, position_ids)
#     return cos, sin

# # GemmaRotaryEmbedding.forward = linear_scaled_forward
# setattr(GemmaRotaryEmbedding, 'forward', linear_scaled_forward)

In [29]:
class GemmaLinearScalingRotaryEmbedding(torch.nn.Module):
    """GemmaRotaryEmbedding extended with linear scaling."""
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()

        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2, dtype=torch.int64).float() / self.dim))
        self.register_buffer("inv_freq", tensor=inv_freq, persistent=False)

    
    def forward(self, x, position_ids, seq_len=0):
        scaling_factor = 4.0
        position_ids = position_ids.float() / scaling_factor
        # cos, sin = super().forward(x, position_ids)
        # return cos, sin
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
        position_ids_expanded = position_ids[:, None, :].float()
        # Force float32 since bfloat16 loses precision on long contexts
        # See https://github.com/huggingface/transformers/pull/29285
        device_type = x.device.type
        device_type = device_type if isinstance(device_type, str) and device_type != "mps" else "cpu"
        with torch.autocast(device_type=device_type, enabled=False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = torch.cat((freqs, freqs), dim=-1)
            cos = emb.cos()
            sin = emb.sin()
        return cos.to(dtype=x.dtype), sin.to(dtype=x.dtype)

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
transformers.models.gemma.modeling_gemma.GemmaRotaryEmbedding = ( 
    GemmaLinearScalingRotaryEmbedding
)

In [31]:
tokenizer = AutoTokenizer.from_pretrained(local_dir)

In [32]:
model = AutoModelForCausalLM.from_pretrained(
    local_dir,
    # device_map="auto",
    torch_dtype=torch.bfloat16
)
model.to(device)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.05s/it]


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaLinearScalingRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (

In [33]:
max_length = model.config.max_position_embeddings
stride = 512
seq_len = encodings.input_ids.size(1)
device = "cuda"

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss
        
    # print(f"nll: {neg_log_likelihood}")
    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
print("perplexity: ", ppl)

  0%|          | 0/565 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.81 GiB. GPU 

In [34]:
!nvidia-smi --gpu-reset -i 0

Error occurred during reset of GPU 00000000:C2:00.0: Unknown Error

1 device did not complete reset successfully, and may be in an unstable state. Please reboot your system.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# perplexity:  tensor(57359.7461, device='cuda:0')
